In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='malgun gothic')

In [2]:
fire119 = pd.read_csv('data/전국 소방서 현황(2018.11.28).csv', encoding='cp949', engine='python')
safe119 = pd.read_csv('data/전국 119안전센터 현황(2018.11.28).csv', encoding='cp949', engine='python')
police112 = pd.read_csv('data/경찰청_경찰관서 위치, 주소 현황_20210924.csv', encoding='cp949', engine='python')

### 여기 부터 소방서 주소정제 시작

In [3]:
li = ['서울특별시','서울특별시','부산광역시','부산광역시','인천광역시','인천광역시','광주광역시','대전광역시',
      '울산광역시','세종특별자치시','경기도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도',
      '제주특별자치도','제주특별자치도','제주특별자치도']

In [ ]:
fire119

In [4]:
fire119_addr = []
for i in fire119['주소']:
    fire119_addr.append(str(i).split(" "))

In [ ]:
fire119_addr

In [5]:
fire119_all_repl = []
for i in range(len(fire119_addr)):
    for j in range(len(li)):
        if fire119_addr[i][0][0] == li[j][0]:
            if fire119_addr[i][0][1] == li[j][1]:
                fire119_addr[i][0] = li[j]
            elif fire119_addr[i][0][1] == li[j][2]:
                fire119_addr[i][0] = li[j]
                break
    fire119_all_repl.append(' '.join(fire119_addr[i]))
fire119_all_repl

['서울특별시 종로구 종로1길 28(수송동)',
 '서울특별시 중구 퇴계로 394(무학동)',
 '서울특별시 광진구 광나루로 480(구의동)',
 '서울특별시 용산구 한강대로 167(한강로2가)',
 '서울특별시 동대문구 장한로 34(장안동)',
 '서울특별시 중랑구 신내로 183(신내동)',
 '서울특별시 영등포구 문래로 197(영등포동4가)',
 '서울특별시 성북구 종암로 27길 3(종암동)',
 '서울특별시 강북구 보건소길 2(번동)',
 '서울특별시 도봉구 도봉로 666(방학동)',
 '서울특별시 노원구 한글비석로 1길 8(하계동)',
 '서울특별시 은평구 통일로 962(진관동)',
 '서울특별시 서대문구 연희로 182(연희동)',
 '서울특별시 마포구 창전로 76(신수동)',
 '서울특별시 강남구 테헤란로 629(삼성동)',
 '서울특별시 서초구 사평대로 67(반포동)',
 '서울특별시 관악구 관악로 97(봉천동)',
 '서울특별시 동작구 여의대방로16길 55(신대방동)',
 '서울특별시 강서구 양천로 550(등촌동)',
 '서울특별시 양천구 목동서로 180(목동)',
 '서울특별시 강동구 성내로 39(성내동)',
 '서울특별시 송파구 오금로51길 56(마천동)',
 '서울특별시 구로구 경인로 408(고척동)',
 '서울특별시 성동구 살곶이길 331(행당동)',
 '부산광역시 중구 중앙대로 110(중앙동4가)',
 '부산광역시 동구 범일로 115(범일동)',
 '부산광역시 연제구 고분로 216(연산동)',
 '부산광역시 사상구 삼덕로 17(삼락동)',
 '부산광역시 사하구 장평로 270(신평동)',
 '부산광역시 해운대구 해운대해변로 106(우동)',
 '부산광역시 금정구 중앙대로 1789(부곡동)',
 '부산광역시 수영구 광일로 64(광안동)',
 '부산광역시 강서구 녹산산단361로 9(송정동)',
 '부산광역시 기장군 정관면 용수로 7',
 '부산광역시 영도구 해양로 297(동삼동)',
 '대구광역시 중구 달구벌대로 1926',
 '대구광역시 동구

In [6]:
fire119['주소(정)'] = fire119_all_repl

In [ ]:
fire119.to_csv('data/fire119.csv', encoding='cp949', index=False)

### 여기까지 소방서 주소정제 끝

### folium 을 이용한 지도화

* 변수 재활용을 위한 삭제

In [7]:
del fire119_addr
del fire119_all_repl

In [8]:
fire119_addr = pd.DataFrame(fire119['주소(정)'].apply(lambda x : x.split()[:2]).tolist(), columns=('시도','군구'))

In [ ]:
fire119_addr

* 나눠놓은 시도와 군구를 시도군구 컬럼으로 만들어서 병합
* count 컬럼을 만들고 값을 0으로 놓는다.

In [9]:
fire119_addr['시도군구'] = fire119_addr.apply(lambda x : x['시도']+' '+x['군구'], axis=1)
fire119_addr['count'] = 0

In [10]:
fire119_group = pd.DataFrame(fire119_addr.groupby(['시도','군구','시도군구'], as_index=False).count())

In [11]:
fire119_group.set_index('시도군구')

,시도,군구,count
시도군구,,,
강원도 강릉시,강원도,강릉시,1
강원도 고성군,강원도,고성군,1
강원도 동해시,강원도,동해시,1
강원도 삼척시,강원도,삼척시,1
강원도 속초시,강원도,속초시,1
...,...,...,...
충청북도 제천시,충청북도,제천시,1
충청북도 증평군,충청북도,증평군,1
충청북도 진천군,충청북도,진천군,1


In [17]:
fire119_group['시도'].value_counts()

경기도        31
서울특별시      24
전라남도       24
경상남도       18
경상북도       17
강원도        16
충청남도       15
부산광역시      11
충청북도       11
인천광역시       8
대구광역시       7
광주광역시       5
대전광역시       4
울산광역시       4
세종특별자치시     2
제주특별자치도     2
Name: 시도, dtype: int64

### 여기부터 119안전센터 주소정제 시작

In [ ]:
safe119.head()

In [ ]:
safe119_addr = []
for i in safe119['주소']:
    safe119_addr.append(str(i).split(" "))

In [ ]:
for i in safe119_addr:
    if i[0] == '수원시' : i.insert(0, '경기도')
    elif i[0] == '성남시' : i.insert(0, '경기도')
    elif i[0] == '파주시' : i.insert(0, '경기도')
    elif i[0] == '양주시' : i.insert(0, '경기도')
    elif i[0] == '가평군' : i.insert(0, '경기도')
    elif i[0] == '충북' : i[0] = '충청북도'
    elif i[0] == '증평군' : i.insert(0, '충청북도')
    elif i[0] == '충남' : i[0] = '충청남도'
    elif i[0] == '전북' : i[0] = '전라북도'
    elif i[0] == '전남' : i[0] = '전라남도'
    elif i[0] == '완도군' : i.insert(0, '전라남도')
    elif i[0] == '함평군' : i.insert(0, '전라남도')
    elif i[0] == '경남' : i[0] = '경상남도'

In [ ]:
safe119_addr

In [ ]:
safe119_all_repl = []
for i in range(len(safe119_addr)):
    safe119_all_repl.append(' '.join(safe119_addr[i]))
safe119_all_repl

In [ ]:
safe119['주소(정)'] = pd.DataFrame(safe119_all_repl)

In [ ]:
safe119.head()

In [ ]:
safe119.to_csv('safe119.csv', encoding='cp949', index=False)

### 여기까지 119안전센터 주소 정제끝

In [ ]:
del safe119_addr
del safe119_all_repl

In [ ]:
safe119_addr = pd.DataFrame(safe119['주소'].apply(lambda y : y.split()[:2]).tolist(), columns=('시도','군구')

### 여기부터 경찰관서

In [ ]:
police112.head()

In [ ]:
police112_addr = []
for i in police112['주소']:
    police112_addr.append(str(i).split(" "))

In [ ]:
police112_addr

In [ ]:
police112_addr_repl = []
for i in range(len(police112_addr)):
    for j in range(len(li)):
        if police112_addr[i][0][0] == li[j][0]:
            if police112_addr[i][0][1] == li[j][1]:
                police112_addr[i][0] = li[j]
            elif police112_addr[i][0][1] == li[j][2]:
                police112_addr[i][0] = li[j]
                break
    police112_addr_repl.append(' '.join(police112_addr[i]))
police112_addr_repl

In [ ]:
police112['주소(정)'] = pd.DataFrame(police112_addr_repl)

In [ ]:
police112.head()

In [ ]:
police112.to_csv('police112.csv', encoding='cp949', index=False)